In [ ]:
!pip install num2words

     |████████████████████████████████| 102kB 6.0MB/s 


In [ ]:
import numpy as np
import pandas as pd 
import re
from num2words import num2words 
from tqdm import tqdm 
import gensim
from gensim.models import KeyedVectors
import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.applications import InceptionV3,Xception
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers.merge import add
from tensorflow.keras.layers import LayerNormalization,Bidirectional,LSTM,Dense,Embedding,RepeatVector,Dropout,Activation,Input,BatchNormalization,GlobalAveragePooling2D,Flatten,Layer,concatenate,Lambda,Add,dot,MaxPooling1D,Conv1D,Multiply,GRU,GlobalAveragePooling1D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,SGD,Adadelta
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cap=pd.read_csv('/content/drive/MyDrive/Image Captioning/Flickr8k_text/Flickr8k.token.txt',sep='\t',header=None)
cap.columns=['image_name','comment']
cap

,image_name,comment
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...
...,...,...
40455,997722733_0cb5439472.jpg#0,A man in a pink shirt climbs a rock face
40456,997722733_0cb5439472.jpg#1,A man is rock climbing high in the air .
40457,997722733_0cb5439472.jpg#2,A person in a red shirt climbing up a rock fac...
40458,997722733_0cb5439472.jpg#3,A rock climber in a red shirt .


In [ ]:
# cap=cap[:100]
# cap

In [ ]:
def seperate_no(text):
    text=text.split('#')
    return text[1]
def seperate_image(text):
    text=text.split('#')
    return text[0]
cap['comment_no']=cap['image_name'].apply(seperate_no)
cap['image_name']=cap['image_name'].apply(seperate_image)
cap

,image_name,comment,comment_no
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,0
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,1
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,2
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,3
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,4
...,...,...,...
40455,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face,0
40456,997722733_0cb5439472.jpg,A man is rock climbing high in the air .,1
40457,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...,2
40458,997722733_0cb5439472.jpg,A rock climber in a red shirt .,3


In [ ]:
cap.isna().sum()

image_name    0
comment       0
comment_no    0
dtype: int64

In [ ]:
cap=cap.drop(cap[cap.image_name=='2258277193_586949ec62.jpg'].index)
cap

,image_name,comment,comment_no
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,0
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,1
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,2
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,3
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,4
...,...,...,...
40455,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face,0
40456,997722733_0cb5439472.jpg,A man is rock climbing high in the air .,1
40457,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...,2
40458,997722733_0cb5439472.jpg,A rock climber in a red shirt .,3


In [ ]:
def preprocessing(text):
  pattern='https?:\S+|[^a-zA-Z0-9]+'
  text=re.sub(pattern,' ',text.lower()).strip()
  text='startseq '+text+' endseq'
  tokens=[]
  for token in text.split():
    try:
      token=num2words(token)
    except:
      pass
    # if token not in stop_words:
#     tokens.append(lemmatizer.lemmatize(token))
    tokens.append(token)
    # tokens.append(stemmer.stem(token))
  return " ".join(tokens)


In [ ]:
cap.comment=cap.comment.apply(preprocessing)

In [ ]:
cap.comment.str.split().apply(len).quantile(0.99)

24.0

In [ ]:
language=list(cap.comment)
tokenizer=Tokenizer()
tokenizer.fit_on_texts(language)

In [ ]:
import pickle

# saving
with open('/content/drive/MyDrive/Image Captioning/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8456

In [ ]:
# c=cap[['comment','image_name']].to_dict('dict')
captions=dict()
# n=len(c['comment'])
# print(n)
img=list(cap.image_name)
com=list(cap.comment)
n=len(img)
for i in range(n):
    # print(i)
    k=img[i]
    v=com[i]
    if k not in captions:
        captions[k]=[]
    captions[k].append(v)


In [ ]:
captions

{'1000268201_693b08cb0e.jpg': ['startseq a child in a pink dress is climbing up a set of stairs in an entry way endseq',
  'startseq a girl going into a wooden building endseq',
  'startseq a little girl climbing into a wooden playhouse endseq',
  'startseq a little girl climbing the stairs to her playhouse endseq',
  'startseq a little girl in a pink dress going into a wooden cabin endseq'],
 '1001773457_577c3a7d70.jpg': ['startseq a black dog and a spotted dog are fighting endseq',
  'startseq a black dog and a tri colored dog playing with each other on the road endseq',
  'startseq a black dog and a white dog with brown spots are staring at each other in the street endseq',
  'startseq two dogs of different breeds looking at each other on the road endseq',
  'startseq two dogs on pavement moving toward each other endseq'],
 '1002674143_1b742ab4b8.jpg': ['startseq a little girl covered in paint sits in front of a painted rainbow with her hands in a bowl endseq',
  'startseq a little 

In [ ]:
img=list(cap.image_name)

In [ ]:
img

['1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg',
 '1002674143_1b742ab4b8.jpg',
 '1002674143_1b742ab4b8.jpg',
 '1002674143_1b742ab4b8.jpg',
 '1002674143_1b742ab4b8.jpg',
 '1002674143_1b742ab4b8.jpg',
 '1003163366_44323f5815.jpg',
 '1003163366_44323f5815.jpg',
 '1003163366_44323f5815.jpg',
 '1003163366_44323f5815.jpg',
 '1003163366_44323f5815.jpg',
 '1007129816_e794419615.jpg',
 '1007129816_e794419615.jpg',
 '1007129816_e794419615.jpg',
 '1007129816_e794419615.jpg',
 '1007129816_e794419615.jpg',
 '1007320043_627395c3d8.jpg',
 '1007320043_627395c3d8.jpg',
 '1007320043_627395c3d8.jpg',
 '1007320043_627395c3d8.jpg',
 '1007320043_627395c3d8.jpg',
 '1009434119_febe49276a.jpg',
 '1009434119_febe49276a.jpg',
 '1009434119_febe49276a.jpg',
 '10094341

In [ ]:
unique_imgs=list(set(img))
len(unique_imgs)

8091

In [ ]:
X_img={}
# for i in tqdm(unique_imgs):
#     if i not in X_img:
#         img_path='/content/drive/MyDrive/Image Captioning/Flicker8k_Dataset/'+str(i)
#         im = load_img(img_path, target_size=(224,224,3))
#         im = img_to_array(im)
#         im=preprocess_input(im)
#         # im=im/255.
#         X_img[i]=im
# np.savez('/content/drive/MyDrive/Image Captioning/inc_img.npz',**X_img)

X_img=np.load('/content/drive/MyDrive/Image Captioning/inc_img.npz')
X_img=dict(X_img)

In [ ]:
val_imgs=unique_imgs[7500:]
X_img_val={}
val_captions={}
for i in val_imgs:
    X_img_val[i]=X_img[i]
    val_captions[i]=captions[i]
    X_img.pop(i)
    captions.pop(i)    



In [ ]:
def data_generator(captions, images, tokenizer, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while True:
        image_ids=list(captions.keys())
        for i in image_ids:
            n+=1
            image=images[i]
            caption_list=captions[i]
            for c in caption_list:
                seq = tokenizer.texts_to_sequences([c])[0]
                
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(image)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                # print('te')
                yield [np.array(X1), np.array(X2)], np.array(y)
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
# X_img['1000268201_693b08cb0e.jpg'].shape

In [ ]:
# X_cap_train
# len(X_img_train)

# y_train

In [ ]:
image_encoder=InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))
image_encoder.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [ ]:
for layer in image_encoder.layers:
        layer.trainable = False

In [ ]:
max_len=25
word2vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/embeddings/crawl-300d-2M-subword.vec', binary=False)
embedding_matrix=np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i]=word2vec.word_vec(word)

In [ ]:
class MultiHeadAttention(Layer):
  def __init__(self,embdim,nheads):
    super(MultiHeadAttention,self).__init__()
    self.embdim=embdim
    self.nheads=nheads
    self.wq=Dense(embdim)
    self.wk=Dense(embdim)
    self.wv=Dense(embdim)
    self.out_dense=Dense(embdim)
    self.depth=embdim//nheads
 
  def split_heads(self,x,batch_size):
    x=tf.reshape(x,(batch_size,-1,self.nheads,self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
 
  def scaled_dot_attention(self,q,k,v):
    # k=(tf.math.square(k))
    # q=(tf.math.square(q))
    qk=tf.matmul(q,k,transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_qk=qk/tf.math.sqrt(dk)
    attention_weights=tf.nn.softmax(scaled_qk,axis=-1)
    output=tf.matmul(attention_weights,v)
    return output,attention_weights


 
  def call(self,inputs):
    q=self.wq(inputs)
    v=self.wv(inputs)
    k=self.wk(inputs)
    batch_size=tf.shape(inputs)[0]
    q=self.split_heads(q,batch_size)
    k=self.split_heads(k,batch_size)
    v=self.split_heads(v,batch_size)
    
    scaled_attention,weights=self.scaled_dot_attention(q,k,v)
    scaled_attention= tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
    concat_attention=tf.reshape(scaled_attention,(batch_size,-1,self.embdim))
    output=self.out_dense(concat_attention)
    return output
 
class PositionalEncoding(Layer):
  def __init__(self, d_model):
    super(PositionalEncoding,self).__init__()
    self.d_model = d_model
 
  def call(self, inputs):
    position = tf.shape(inputs)[1]
 
    position_dims = tf.range(position)[:, tf.newaxis]
    embed_dims = tf.range(self.d_model)[tf.newaxis, :]
    angle_rates = 1 / tf.pow(
        10000.0, tf.cast(
            (2 * (embed_dims // 2)) / self.d_model, tf.float32))
    angle_rads = tf.cast(position_dims, tf.float32) * angle_rates
 
    sines = tf.sin(angle_rads[:, 0::2])
    cosines = tf.cos(angle_rads[:, 1::2]) 
    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)
 
 
 
class Transformer(Layer):
  def __init__(self,embdim,nheads,dff,rate):
    super(Transformer,self).__init__()
    self.ffn1=Dense(dff, activation='relu')
    self.ffn3=Dense(dff/2,activation='relu')
    self.ffn4=Dense(dff/4,activation='relu')
    self.ffn2=Dense(embdim)
    self.mha=MultiHeadAttention(embdim,nheads)
    self.layernorm1 = LayerNormalization(epsilon=1e-6)
    self.layernorm2 = LayerNormalization(epsilon=1e-6)
    self.dropout1 = Dropout(rate)
    self.dropout2 = Dropout(rate)
    self.dropout3 = Dropout(rate)
    self.dropout4 = Dropout(rate)
    
 
  def call(self,inputs):
    att_out=self.mha(inputs)
    att_out=self.dropout1(att_out)
    out1=self.layernorm1(inputs+att_out)
    ff=self.ffn1(out1)
    ff=self.ffn2(ff)
    ff=self.dropout2(ff)
    return self.layernorm2(out1+ff)


class MultiHeadAttention(Layer):
  def __init__(self,embdim,nheads):
    super(MultiHeadAttention,self).__init__()
    self.embdim=embdim
    self.nheads=nheads
    self.wq=Dense(embdim)
    self.wk=Dense(embdim)
    self.wv=Dense(embdim)
    self.out_dense=Dense(embdim)
    self.depth=embdim//nheads
 
  def split_heads(self,x,batch_size):
    x=tf.reshape(x,(batch_size,-1,self.nheads,self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
 
  def scaled_dot_attention(self,q,k,v):
    # k=(tf.math.square(k))
    # q=(tf.math.square(q))
    qk=tf.matmul(q,k,transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_qk=qk/tf.math.sqrt(dk)
    attention_weights=tf.nn.softmax(scaled_qk,axis=-1)
    output=tf.matmul(attention_weights,v)
    return output,attention_weights


 
  def call(self,inputs):
    q=self.wq(inputs)
    v=self.wv(inputs)
    k=self.wk(inputs)
    batch_size=tf.shape(inputs)[0]
    q=self.split_heads(q,batch_size)
    k=self.split_heads(k,batch_size)
    v=self.split_heads(v,batch_size)
    
    scaled_attention,weights=self.scaled_dot_attention(q,k,v)
    scaled_attention= tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
    concat_attention=tf.reshape(scaled_attention,(batch_size,-1,self.embdim))
    output=self.out_dense(concat_attention)
    return output
 
class PositionalEncoding(Layer):
  def __init__(self, d_model):
    super(PositionalEncoding,self).__init__()
    self.d_model = d_model
 
  def call(self, inputs):
    position = tf.shape(inputs)[1]
 
    position_dims = tf.range(position)[:, tf.newaxis]
    embed_dims = tf.range(self.d_model)[tf.newaxis, :]
    angle_rates = 1 / tf.pow(
        10000.0, tf.cast(
            (2 * (embed_dims // 2)) / self.d_model, tf.float32))
    angle_rads = tf.cast(position_dims, tf.float32) * angle_rates
 
    sines = tf.sin(angle_rads[:, 0::2])
    cosines = tf.cos(angle_rads[:, 1::2]) 
    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)
 
 
 
class Transformer(Layer):
  def __init__(self,embdim,nheads,dff,rate):
    super(Transformer,self).__init__()
    self.ffn1=Dense(dff, activation='relu')
    self.ffn3=Dense(dff/2,activation='relu')
    self.ffn4=Dense(dff/4,activation='relu')
    self.ffn2=Dense(embdim)
    self.mha=MultiHeadAttention(embdim,nheads)
    self.layernorm1 = LayerNormalization(epsilon=1e-6)
    self.layernorm2 = LayerNormalization(epsilon=1e-6)
    self.dropout1 = Dropout(rate)
    self.dropout2 = Dropout(rate)
    self.dropout3 = Dropout(rate)
    self.dropout4 = Dropout(rate)
    
 
  def call(self,inputs):
    att_out=self.mha(inputs)
    att_out=self.dropout1(att_out)
    out1=self.layernorm1(inputs+att_out)
    ff=self.ffn1(out1)
    ff=self.ffn2(ff)
    ff=self.dropout2(ff)
    return self.layernorm2(out1+ff)



class attention(Layer):
  def __init__(self,dim):
    super(attention,self).__init__()
    self.W=Dense(dim)
    self.W2=Dense(dim)

  def call(self,features,prev):
    prev=tf.expand_dims(prev, 1)
    prev=self.W2(prev)
    c=self.W(features)
    c=tf.math.tanh(c+prev)
    # print(c.shape)
    aw=tf.nn.softmax(c,axis=1)   
    q=aw*c
    return q


In [ ]:
embdim=300
nheads=10
dff=512
rate=0.2
nt=3
embedding_layer=Embedding(vocab_size,300,weights=[embedding_matrix],input_length=max_len,trainable=False)
transform1=Transformer(embdim,nheads,dff,rate)
transform2=Transformer(embdim,nheads,dff,rate)
att=attention(300)
X_i=image_encoder.output
# X_i = GlobalAveragePooling2D()(X_i)
X_i=tf.reshape(X_i, (tf.shape(X_i)[0], -1, 2048))
# print(X_i.shape)
# X_i=RepeatVector(max_len)(X_i)
# print(X_i.shape)

# X_i=transform1(X_i)
X_cap_i=Input(shape=(max_len,))
X_cap=embedding_layer(X_cap_i)
# for i in range(nt):
prev=LSTM(300)(X_cap)
X_i=att(X_i,prev)
# X_cap=Flatten()(X_cap)
# X_cap=Dense(256,activation='relu')(X_cap)
dec = concatenate([X_i, X_cap],axis=-2)
# dec = Dense(256, activation='relu')(dec)
dec=Dropout(0.5)(dec)
# dec=Bidirectional(LSTM(256, return_sequences=False))(dec)
dec= LSTM(300)(dec)
# dec= GlobalAveragePooling1D()(dec)
out = Dense(vocab_size, activation='softmax')(dec)
model = Model(inputs=[image_encoder.input, X_cap_i], outputs=out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
X_i=image_encoder.output
# X_i= Dense(256, activation='relu')(X_i)
X_i = GlobalAveragePooling2D()(X_i)
X_i= Dense(256, activation='relu')(X_i)
X_i=RepeatVector(max_len)(X_i)

embdim=300
nheads=10
dff=512
rate=0.2
embedding_layer=Embedding(vocab_size,300,weights=[embedding_matrix],input_length=max_len,trainable=False)
transform=Transformer(embdim,nheads,dff,rate)

X_cap_i=Input(shape=(max_len,))
X_cap=embedding_layer(X_cap_i)
X_cap=transform(X_cap)
# X_cap=Flatten()(X_cap)
# X_cap=Dense(256,activation='relu')(X_cap)
dec = concatenate([X_i, X_cap])
# dec = Dense(256, activation='relu')(dec)
dec=Dropout(0.5)(dec)
dec=Conv1D(256, 3, activation='relu')(dec)
dec=MaxPooling1D()(dec)
dec=GlobalAveragePooling1D()(dec)
out = Dense(vocab_size, activation='softmax')(dec)
model = Model(inputs=[image_encoder.input, X_cap_i], outputs=out)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [ ]:
# lr_decay = (1./0.9 -1)/bs
opt = Adam(lr=(1e-4),beta_1=0.9,beta_2=0.999)
loss=tf.keras.losses.CategoricalCrossentropy()
callbacks = [ModelCheckpoint('./image_cap.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1),
             EarlyStopping(monitor='val_loss', patience=4, verbose=1, min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, cooldown=0, min_lr=1e-7, verbose=1)]
# opt=SGD(lr=1e-4)
# opt=Adadelta(learning_rate=1e-2)
# model.load_weights('/content/drive/MyDrive/Image Captioning/model_t_weights_2_17.h5')
model.compile(optimizer=opt, loss=loss ,metrics=['accuracy'])

In [ ]:
number_pics_per_batch = 3
steps = len(captions)//number_pics_per_batch
val_steps=len(val_captions)//number_pics_per_batch
train_generator=data_generator(captions,X_img,tokenizer,max_len,number_pics_per_batch)
val_generator=data_generator(val_captions,X_img_val,tokenizer,max_len,number_pics_per_batch)
model.fit(train_generator,validation_data=val_generator,epochs=20,steps_per_epoch=steps,validation_steps=val_steps,callbacks=callbacks)


In [ ]:
model.save_weights("/content/drive/MyDrive/Image Captioning/model_weights3_20.h5")
